# Net Demand

`Net Demand` = `Total Demand` - `Vesting Contract`
             = `Demand` + `TCL` + `Transmisson Loss` - `Vesting Contract`

In [1]:
import pandas as pd

rt_dpr = pd.read_csv('./data/RT_DPR.csv')
rt_dpr = rt_dpr[['Date', 'Period', 'Demand', 'TCL', 'TransmissionLoss']]
rt_dpr['TransmissionLoss'] = rt_dpr['TransmissionLoss'].fillna(0)
rt_dpr = rt_dpr[rt_dpr['Date'] > '2023-06-30']
rt_dpr = rt_dpr.sort_values(by=['Date', 'Period'])
rt_dpr.reset_index(drop=True, inplace=True)
rt_dpr.head(1)

,Date,Period,Demand,TCL,TransmissionLoss
0,2023-07-01,1,6086.883,0.0,32.68


In [2]:
vc_per = pd.read_csv('./data/VCData_Period.csv')

Create a view of [Total Demand, TCQ, Net Demand] from `rt_dpr` and `vc_per`.

In [3]:
rt_dpr['Total Demand'] = rt_dpr['Demand'] + rt_dpr['TCL'] + rt_dpr['TransmissionLoss']
view = rt_dpr[['Date', 'Period', 'Total Demand']].copy()

In [4]:
import holidays

sg_holidays = holidays.country_holidays('SG')

In [5]:
import datetime as dt
def find_tcq(row):
    # print(row)
    date_obj = dt.datetime.strptime(row['Date'], '%Y-%m-%d')
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    
    isWeekend = 1 if date_obj.isoweekday() > 5 else 0
    isPublicHoliday = date_obj in sg_holidays
    
    if isWeekend or isPublicHoliday:
        return vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter)]['TCQ_Weekend_PH'].values[0] / 1000
    else:
        return vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter)]['TCQ_Weekday'].values[0] / 1000

In [6]:
view['TCQ'] = view.apply(lambda row: find_tcq(row), axis=1)
view['Net Demand'] = view['Total Demand'] - view['TCQ']
view.tail(96)

,Date,Period,Total Demand,TCQ,Net Demand
12801,2024-03-23,34,6491.688,415.350538,6076.337462
12802,2024-03-23,35,6492.116,415.350538,6076.765462
12803,2024-03-23,36,6544.045,415.350538,6128.694462
12804,2024-03-23,37,6628.170,415.350538,6212.819462
12805,2024-03-23,38,6731.825,415.350538,6316.474462
...,...,...,...,...,...
12892,2024-03-25,29,6979.539,355.818726,6623.720274
12893,2024-03-25,30,7042.605,355.818726,6686.786274
12894,2024-03-25,31,7067.406,355.818726,6711.587274
12895,2024-03-25,32,7121.950,355.818726,6766.131274


In [7]:
view.to_csv('./data/NET.csv', index=False)

In [8]:
view.tail(1)

,Date,Period,Total Demand,TCQ,Net Demand
12896,2024-03-25,33,7117.993,355.818726,6762.174274
